## Creating a logistic regression model in Python!

In [1]:
# Reference: https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Logistic%20Regression%20balanced.ipynb

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14) 

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from mpl_toolkits.basemap import Basemap

import os
import datetime
import pytz
import re

import peakutils
import statsmodels.api as sm

import requests

#Read data from a local csv file:

##Will change this to scrape files from the Smartfin.org website later.
data = pd.read_csv('Motion_13735.CSV', header=0)   
data = data.dropna()

#Print out the column headings:
print(data.shape)
print(list(data.columns))

(0, 13)
['UTC', 'Time', 'IMU A1', 'IMU A2', 'IMU A3', 'IMU G1', 'IMU G2', 'IMU G3', 'IMU M1', 'IMU M2', 'IMU M3', 'Latitude', 'Longitude']


In [3]:
ride_ids = ['14827']


#ride_ids = ['14827']
# 14743 - Motion Control July 10th
# 14750 - Magnetometer Control July 11th
# 14814 - Pool Displacement Control July 17th
# 14815 - Compass Orientation (Lying on Charger Side) July 19th
# 14816 - Orientation w Higher Sampling (Lying on Charger Side) July 20th
# 14827 - Pool Displacement Control w Higher Sampling (Jul 23)
# 14888 - First Buoy Calibration Experiment (July 30)
# 15218 - Jasmine's Second Ride Sesh filmed with GoPro (Aug 29)

In [4]:
#%% Fin ID scraper
# Input fin ID, get all ride IDs
# base URL to which we'll append given fin IDs
fin_url_base = 'http://surf.smartfin.org/fin/'

# Look for the following text in the HTML contents in fcn below
str_id_ride = 'rideId = \'' # backslash allows us to look for single quote
str_id_date = 'var date = \'' # backslash allows us to look for single quote

#%% Ride ID scraper
# Input ride ID, get ocean and motion CSVs
# Base URL to which we'll append given ride IDs
ride_url_base = 'https://surf.smartfin.org/ride/'

# Look for the following text in the HTML contents in fcn below
str_id_csv = 'img id="temperatureChart" class="chart" src="' 

def get_csv_from_ride_id(rid):
    # Build URL for each individual ride
    ride_url = ride_url_base+str(rid)
    print(ride_url)
    
    # Get contents of ride_url
    html_contents = requests.get(ride_url).text
    
    # Find CSV identifier 
    loc_csv_id = html_contents.find(str_id_csv)
    
    # Different based on whether user logged in with FB or Google
    offset_googleOAuth = [46, 114]
    offset_facebkOAuth = [46, 112]
    if html_contents[loc_csv_id+59] == 'f': # Facebook login
        off0 = offset_facebkOAuth[0]
        off1 = offset_facebkOAuth[1]
    else: # Google login
        off0 = offset_googleOAuth[0]
        off1 = offset_googleOAuth[1]
        
    csv_id_longstr = html_contents[loc_csv_id+off0:loc_csv_id+off1]
    
#    print(csv_id_longstr)
    
    # Stitch together full URL for CSV
    if ("media" in csv_id_longstr) & ("Calibration" not in html_contents): # other junk URLs can exist and break everything
        
        ocean_csv_url = 'https://surf.smartfin.org/'+csv_id_longstr+'Ocean.CSV'
        motion_csv_url = 'https://surf.smartfin.org/'+csv_id_longstr+'Motion.CSV'
        
        print(ocean_csv_url)
        # Go to ocean_csv_url and grab contents (theoretically, a CSV)
        ocean_df_small = pd.read_csv(ocean_csv_url, parse_dates = [0])
        elapsed_timedelta = (ocean_df_small['UTC']-ocean_df_small['UTC'][0])
        ocean_df_small['elapsed'] = elapsed_timedelta/np.timedelta64(1, 's')
        
        motion_df_small = pd.read_csv(motion_csv_url, parse_dates = [0])
        
        # Reindex on timestamp if there are at least a few rows
        if len(ocean_df_small) > 1:
            ocean_df_small.set_index('UTC', drop = True, append = False, inplace = True)
            motion_df_small.set_index('UTC', drop = True, append = False, inplace = True)
            
            #print(ocean_df_small)
            #print(motion_df_small)
            
            #May need to change this sampling interval:
            sample_interval = '33ms'
            
            
            ocean_df_small_resample = ocean_df_small.resample(sample_interval).mean()
            motion_df_small_resample = motion_df_small.resample(sample_interval).mean()
            
            # No need to save many extra rows with no fix
            motion_df_small = motion_df_small[~np.isnan(motion_df_small.Latitude)]
            
            return ocean_df_small_resample, motion_df_small_resample

    else:
        ocean_df_small_resample = pd.DataFrame() # empty DF just so something is returned
        motion_df_small_resample = pd.DataFrame() 
        return ocean_df_small_resample, motion_df_small_resample
    
appended_ocean_list = [] # list of DataFrames from original CSVs
appended_motion_list = []
appended_multiIndex = [] # fin_id & ride_id used to identify each DataFrame

## Nested loops (for each fin ID, find all ride IDs, then build a DataFrame from all ride CSVs)
## (Here, ride IDS are either ocean or motion dataframes)
count_good_fins = 0
    
# Loop over ride_ids and find CSVs
for rid in ride_ids:
    try:
        new_ocean_df, new_motion_df = get_csv_from_ride_id(rid) # get given ride's CSV from its ride ID using function above
        #print(len(new_ocean_df))
        #print(len(new_motion_df))
        if not new_ocean_df.empty: # Calibration rides, for example
            # Append only if DF isn't empty. There may be a better way to control empty DFs which are created above
            appended_multiIndex.append(str(rid)) # build list to be multiIndex of future DataFrame
            appended_ocean_list.append(new_ocean_df)
            appended_motion_list.append(new_motion_df)
            print("Ride data has been uploaded.")
            #print("Ride: ", rid, "data has been uploaded.")
            count_good_fins += 1
        
    except: 
        print("Ride threw an exception!")
        #print("Ride ", rid, "threw an exception!")    

#%% Build the "Master" DataFrame

# appended_ocean_df.summary()
df_keys = tuple(appended_multiIndex) # keys gotta be a tuple, a list which data in it cannot be changed
ocean_df = pd.concat(appended_ocean_list, keys = df_keys, names=['ride_id'])
motion_df = pd.concat(appended_motion_list, keys = df_keys, names = ['ride_id'])


##Here, maybe just use info from the motion_df and don't worry about ocean_df data for now.
##If you do want ocean_df data, look at how Phil was getting it from "July 10th and 11th Calibration" jupyter notebook file.

print(motion_df)

https://surf.smartfin.org/ride/14827
https://surf.smartfin.org/media/201807/google_117589279598321562176_000666D321BE_180723164430_Ocean.CSV
Ride data has been uploaded.
                                         Time  IMU A1  IMU A2  IMU A3  IMU G1  \
ride_id UTC                                                                     
14827   2018-07-23 16:44:42.915  1.370666e+09   138.0  -380.0   333.0 -1115.0   
        2018-07-23 16:44:42.948  1.370666e+09   181.0  -459.0   225.0  -720.0   
        2018-07-23 16:44:42.981  1.370666e+09   108.0  -446.0   291.0  -976.0   
        2018-07-23 16:44:43.014  1.370666e+09   133.0  -401.0   275.0  -558.0   
        2018-07-23 16:44:43.047  1.370666e+09   153.0  -415.0   101.0  -799.0   
        2018-07-23 16:44:43.080  1.370666e+09    46.0  -435.0   307.0  -760.0   
        2018-07-23 16:44:43.113  1.370666e+09   243.0  -324.0   412.0  -247.0   
        2018-07-23 16:44:43.146  1.370666e+09   281.0  -382.0   311.0   -69.0   
        2018-07-23 1

In [5]:
#Drop the latitude and longitude values since most of them are Nan:
motion_df_dropped = motion_df.drop(columns=['Latitude', 'Longitude'])
print(motion_df_dropped)


                                         Time  IMU A1  IMU A2  IMU A3  IMU G1  \
ride_id UTC                                                                     
14827   2018-07-23 16:44:42.915  1.370666e+09   138.0  -380.0   333.0 -1115.0   
        2018-07-23 16:44:42.948  1.370666e+09   181.0  -459.0   225.0  -720.0   
        2018-07-23 16:44:42.981  1.370666e+09   108.0  -446.0   291.0  -976.0   
        2018-07-23 16:44:43.014  1.370666e+09   133.0  -401.0   275.0  -558.0   
        2018-07-23 16:44:43.047  1.370666e+09   153.0  -415.0   101.0  -799.0   
        2018-07-23 16:44:43.080  1.370666e+09    46.0  -435.0   307.0  -760.0   
        2018-07-23 16:44:43.113  1.370666e+09   243.0  -324.0   412.0  -247.0   
        2018-07-23 16:44:43.146  1.370666e+09   281.0  -382.0   311.0   -69.0   
        2018-07-23 16:44:43.179  1.370666e+09   217.0  -412.0   135.0   323.0   
        2018-07-23 16:44:43.212  1.370666e+09   341.0  -546.0    56.0   190.0   
        2018-07-23 16:44:43.

In [6]:
#Sync the video footage with the UTC time from the Smartfin. 
#Code to convert UTC Time to PCT time (UTC Time is 7 hrs. ahead of PCT Time):

#A reference: https://stackoverflow.com/questions/22800079/converting-time-zone-pandas-dataframe


#sLength = len(motion_df_dropped['UTC'])
#motion_df_dropped['PCT'] = p.Series(np.random.randn(sLength), index=motion_df_dropped.index)


#motion_df_dropped['PCT'] = (motion_df_dropped.UTC)

from datetime import datetime, timedelta
from pytz import timezone
import pytz

#To get the first column i.e. 'Time': 
#print(motion_df_dropped.iloc[:,0])


pacific = pytz.timezone('US/Pacific')
#motion_df_dropped.index = motion_df_dropped.index.tz_localize(pytz.utc).tz_convert(pacific)
#motion_df_dropped.index = motion_df_dropped.index.tz_localize(pytz.utc).tz_convert(pacific)

df = pd.DataFrame(motion_df_dropped)
df.index = df.index.tz_localize('UTC')
df.index = df.index.tz_convert(pacific)


AttributeError: 'MultiIndex' object has no attribute 'tz_localize'

In [10]:
#Create an elapsed_timedelta field:

#timedelta_values = (motion_df_dropped['Time']-motion_df_dropped['Time'][0])
#motion_df_dropped.insert(loc=1, column='TimeDelta', value=timedelta_values, drop=True)
motion_df_dropped['TimeDelta'] = (motion_df_dropped['Time']-motion_df_dropped['Time'][0])
#print(elapsed_timedelta)
#motion_df_dropped.head()
motion_df_dropped.head(10)

Time  TimeDelta  IMU A1  IMU A2  \
ride_id UTC                                                                
14827   2018-07-23 16:44:42.915  1.370666e+09        0.0   138.0  -380.0   
        2018-07-23 16:44:42.948  1.370666e+09       33.0   181.0  -459.0   
        2018-07-23 16:44:42.981  1.370666e+09       66.0   108.0  -446.0   
        2018-07-23 16:44:43.014  1.370666e+09       99.0   133.0  -401.0   
        2018-07-23 16:44:43.047  1.370666e+09      132.0   153.0  -415.0   
        2018-07-23 16:44:43.080  1.370666e+09      165.0    46.0  -435.0   
        2018-07-23 16:44:43.113  1.370666e+09      198.0   243.0  -324.0   
        2018-07-23 16:44:43.146  1.370666e+09      231.0   281.0  -382.0   
        2018-07-23 16:44:43.179  1.370666e+09      265.0   217.0  -412.0   
        2018-07-23 16:44:43.212  1.370666e+09      297.0   341.0  -546.0   

                                 IMU A3  IMU G1  IMU G2  IMU G3  IMU M1  \
ride_id UTC                                                               
14827   2018-07-23 16:44:42.915   333.0 -1115.0   622.0  -484.0   215.0   
        2018-07-23 16:44:42.948   225.0  -720.0   405.0  -962.0   199.0   
        2018-07-23 16:44:42.981   291.0  -976.0   184.0  -838.0   191.0   
        2018-07-23 16:44:43.014   275.0  -558.0   612.0  -420.0   196.0   
        2018-07-23 16:44:43.047   101.0  -799.0   228.0  -482.0   194.0   
        2018-07-23 16:44:43.080   307.0  -760.0    59.0   -19.0   185.0   
        2018-07-23 16:44:43.113   412.0  -247.0   240.0   111.0   191.0   
        2018-07-23 16:44:43.146   311.0   -69.0  -225.0   -36.0   188.0   
        2018-07-23 16:44:43.179   135.0   323.0  -760.0   -48.0   192.0   
        2018-07-23 16:44:43.212    56.0   190.0 -1010.0    49.0   175.0   

                                 IMU M2  IMU M3  
ride_id UTC                                      
14827   2018-07-23 16:44:42.915   315.0    83.0  
        2018-07-23 16:44:42.948   331.0    93.0  
        2018-07-23 16:44:42.981   347.0   107.0  
        2018-07-23 16:44:43.014   352.0   108.0  
        2018-07-23 16:44:43.047   358.0   126.0  
        2018-07-23 16:44:43.080   365.0   133.0  
        2018-07-23 16:44:43.113   375.0   141.0  
        2018-07-23 16:44:43.146   374.0   144.0  
        2018-07-23 16:44:43.179   370.0   136.0  
        2018-07-23 16:44:43.212   371.0   135.0

In [17]:
#Create a "surf" label, label each individual line of data from reviewing surf videotape: 

#Creates a surf array, with all values = 0:
sLength = len(motion_df_dropped['Time'])
surf_array = [0] * sLength
motion_df_dropped['surf'] = pd.Series(surf_array, index=motion_df_dropped.index)

#Creates a surf array, where we can now choose values based on the TimeDelta,
#this will be useful for labelling data from our videotaped sessions:
surf_array = []
for td in motion_df_dropped['TimeDelta']:
    if td > 66.0:
        surf_array.append(1)
    else:
        surf_array.append(0)
        
motion_df_dropped['surf'] = pd.Series(surf_array, index=motion_df_dropped.index)
        

motion_df_dropped.head()



Time  TimeDelta  IMU A1  IMU A2  \
ride_id UTC                                                                
14827   2018-07-23 16:44:42.915  1.370666e+09        0.0   138.0  -380.0   
        2018-07-23 16:44:42.948  1.370666e+09       33.0   181.0  -459.0   
        2018-07-23 16:44:42.981  1.370666e+09       66.0   108.0  -446.0   
        2018-07-23 16:44:43.014  1.370666e+09       99.0   133.0  -401.0   
        2018-07-23 16:44:43.047  1.370666e+09      132.0   153.0  -415.0   

                                 IMU A3  IMU G1  IMU G2  IMU G3  IMU M1  \
ride_id UTC                                                               
14827   2018-07-23 16:44:42.915   333.0 -1115.0   622.0  -484.0   215.0   
        2018-07-23 16:44:42.948   225.0  -720.0   405.0  -962.0   199.0   
        2018-07-23 16:44:42.981   291.0  -976.0   184.0  -838.0   191.0   
        2018-07-23 16:44:43.014   275.0  -558.0   612.0  -420.0   196.0   
        2018-07-23 16:44:43.047   101.0  -799.0   228.0  -482.0   194.0   

                                 IMU M2  IMU M3  surf  
ride_id UTC                                            
14827   2018-07-23 16:44:42.915   315.0    83.0     0  
        2018-07-23 16:44:42.948   331.0    93.0     0  
        2018-07-23 16:44:42.981   347.0   107.0     0  
        2018-07-23 16:44:43.014   352.0   108.0     1  
        2018-07-23 16:44:43.047   358.0   126.0     1

In [ ]:
#If we ever need to calculate averages from data rows/columns here's a link to do that:
#https://stackoverflow.com/questions/31698861/add-column-to-the-end-of-pandas-dataframe-containing-average-of-previous-data

